In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"

import numpy as np
import sklearn

from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.datasets import make_regression
from sklearn.decomposition import SparseCoder, sparse_encode
from sklearn.linear_model import orthogonal_mp_gram

import scipy.linalg as LA
from scipy.sparse.linalg import lsqr, lsmr

In [2]:
eye = np.eye(3)

In [3]:
eye

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [4]:
eye[0:1]

array([[1., 0., 0.]])

In [5]:
eye[0:1].shape

(1, 3)

In [6]:
eye[0:1].T.shape

(3, 1)

In [7]:
eye[[0, 1]]

array([[1., 0., 0.],
       [0., 1., 0.]])

# Solve

In [8]:
def OMP(Y, T_0, D, rng=42, debug=False):
    # loss = np.empty(num_iter)
    # rng = np.random.default_rng(rng)

    X = np.zeros((Y.shape[0], D.shape[0]))

    # gram = D @ D.T
    # cov = D @ Y.T
    # X = sparse_encode(Y, 
    #                   D, 
    #                   algorithm='omp', 
    #                   n_nonzero_coefs = T_0,
    #                   gram = gram,
    #                   cov = cov,
    #                   copy_cov=False)

    for i, y in enumerate(Y):
        I = []
        D_I = np.zeros((T_0, D.shape[1]))
        r = y
        gamma = 0

        for j in range(T_0):
            D_r = np.abs(D @ r)
            k = np.argmax(D_r)
            
            I.append(k)
            D_I[j] = D[k]
            
            # gamma, res, rank, s = LA.lstsq(D_I[:j+1].T, y, lapack_driver='gelsd')
            # gamma, istop, itn, res = lsmr(D_I[:j+1].T, y)[:4]
            gamma = np.linalg.solve(D_I[:j+1] @ D_I[:j+1].T, D_I[:j+1] @ y)
            if debug:
                print(gamma.shape)
                print(D_I[:j+1].T.shape)
                print(y.shape)
            r = y - D_I[:(j+1)].T @ gamma 
            if debug:
                print(np.sum(r * r))
                # print(res)

        X[i, I] = gamma
        
    return X

# Handwritten QR (Gram-Schmidt)

In [9]:
def OMP_2(Y, T_0, D, rng=42, debug=False):
    # loss = np.empty(num_iter)
    # rng = np.random.default_rng(rng)

    X = np.zeros((Y.shape[0], D.shape[0]))
    D_norm = np.linalg.norm(D, axis=1)

    for i, y in enumerate(Y):
        I = []
        D_I = np.zeros((T_0, D.shape[1]))
        r = y
        gamma = 0
        Q = np.empty((D.shape[1], T_0))
        R = np.empty((T_0, T_0))
        
        for j in range(T_0):
            D_r = np.abs(D @ r)
            k = np.argmax(D_r)
            
            I.append(k)
            D_I[j] = D[k]

            if j == 0:
                # Q, R = LA.qr(D[k][:, np.newaxis],
                #             overwrite_a = False,
                #             mode = 'economic')
                # D_k_norm = np.linalg.norm(D[k])
                D_k_norm = D_norm[k]
                R[0, 0] = D_k_norm
                Q[:, 0] = D[k] / D_k_norm
                
                if debug:
                    print(np.linalg.norm(D_k_norm))
                    print(Q[:, 0].T @ Q[:, 0])
                    print(R)

                gamma = (y @ Q[:, :j+1]) / D_k_norm
                
            else:
                # Q, R = LA.qr_insert(Q, R, D[k][:, np.newaxis], j, which='col') 
                dot = Q[:, :j].T @ D[k]
                R[0:j, j] = dot
                q_j = D[k] - Q[:, :j] @ dot

                q_j_norm = np.linalg.norm(q_j)
                R[j, j] = q_j_norm
                Q[:, j] = q_j / q_j_norm

                if debug:            
                    print(Q[:, :j+1].T.shape)
                    print(y.shape)
                gamma = LA.solve_triangular(R[:j+1, :j+1], 
                                        Q[:, :j+1].T @ y, 
                                        overwrite_b = True,
                                        check_finite = False)
                

            if debug:
                print(gamma.shape)
                print(D_I[:j+1].T.shape)
                print(y.shape)
            r = y - gamma @ D_I[:(j+1)]  
            if debug:
                print(np.sum(r * r))
                # print(res)

        X[i, I] = gamma
        
    return X

# Batched OMP

In [10]:
def OMP_3(Y, T_0, D, batch_size = 2, rng=42, debug=False):
    # loss = np.empty(num_iter)
    # rng = np.random.default_rng(rng)

    X = np.zeros((Y.shape[0], D.shape[0]))
    D_norm = np.linalg.norm(D, axis=1, keepdims=True)

    splits = np.arange(0, Y.shape[0], step=batch_size)
    Y_batches = np.split(Y, splits)[1:]
    
    for i, y in enumerate(Y_batches):
        if i == (len(Y_batches) - 1):
            batch_size = np.arange(splits[i], len(Y)).shape[0]
            if debug:
                print(f'batch_size = {batch_size}')
        I = np.empty((batch_size, T_0), dtype=np.int32)
        D_I = np.zeros((batch_size, T_0, D.shape[1]))
        r = y   # (batch_size, D.shape[1])
        gamma = 0
        Q = np.empty((batch_size, D.shape[1], T_0))
        R = np.empty((batch_size, T_0, T_0))
        
        for j in range(T_0):
            if debug:
                print(f'Batch {i}, Iteration {j}')
                print(f'r shape: {r.shape}')
            D_r = np.abs(r @ D.T)
            k = np.argmax(D_r, axis = 1)
            
            I[:, j] = k
            D_I[:, j] = D[k]

            if j == 0:
                D_k_norm = D_norm[k]
                R[:, 0, 0] = D_k_norm[:, 0]
                Q[:, :, 0] = D[k] / D_k_norm
                
                if debug:
                    print(D_k_norm)
                    print(np.sum(Q[:, :, 0] ** 2, axis = 1))
                    print(R[:, 0, 0])

                gamma = np.squeeze(y[:, np.newaxis] @ Q[:, :, :1], axis=1) 
                gamma /= D_k_norm
                
            else:
                if debug:
                    print(np.transpose(Q[:, :, :j], (0, 2, 1)).shape)
                    print(D[k][..., np.newaxis].shape)
                # dot = np.transpose(Q[:, :, :j], (0, 2, 1)) @ D[k][..., np.newaxis]
                dot = np.squeeze(D[k][:, np.newaxis] @ Q[:, :, :j], axis=1)
                
                # R[:, 0:j, j] = dot[0]
                R[:, 0:j, j] = dot
                
                # q_j = D[k] - (Q[:, :, :j] @ dot)[..., 0]
                q_j = D[k] - np.squeeze(Q[:, :, :j] @ dot[..., np.newaxis], axis = -1)
                
                # q_j_norm = np.linalg.norm(q_j)
                q_j_norm = np.linalg.norm(q_j, axis = 1)
                
                R[:, j, j] = q_j_norm
                # Q[:, :, j] = q_j / q_j_norm
                Q[:, :, j] = q_j / q_j_norm[:, np.newaxis]
                

                if debug:            
                    print(Q[:, :, :j+1].shape)
                    print(y.shape)
                Q_T_y = np.transpose(y[:, np.newaxis] @ Q[:, :, :j+1], (0, 2, 1))
                gamma = LA.solve_triangular(R[:, :j+1, :j+1], 
                                        Q_T_y, 
                                        overwrite_b = True,
                                        check_finite = False)
                if debug:
                    print(f'original gamma shape: {gamma.shape}')
                # gamma = np.transpose(gamma, (0, 2, 1))
                gamma = np.squeeze(gamma, axis=-1)
                

            if debug:
                print(gamma.shape)
                print(f'D_I shape: {D_I[:, :j+1].shape}')
                print(f'y shape: {y.shape}')
            # est = (np.transpose(D_I[:, :j+1], (0, 2, 1)) @ gamma[..., np.newaxis])[..., 0] 
            est = np.squeeze(gamma[:, np.newaxis] @ D_I[:, :j+1], axis=1)
            if debug:
                print(f'est shape: {est.shape}')
            r = y - est
            # if r.ndim > 2:
                # r = np.transpose(r, (0, 2, 1))
            if debug:
                print(r.shape)
            if debug:
                print(np.sum(r * r, axis = -1))
                # print(res)

        # X[i, I] = gamma
        if i == (len(Y_batches) - 1):
            if debug:
                # print(splits[i].dtype)
                # print(I.dtype)
                print(f'gamma: {gamma}')
                print(f'gamma shape: {gamma.shape}')
                print(f'k shape: {k.shape}')
                print(f'I shape: {I.shape}')
            # if T_0 == 1:
            #     X[np.arange(splits[i], len(Y)), I[:, 0]] = gamma[:, 0]
            # else:
            #     X[np.arange(splits[i], len(Y))[:, np.newaxis], I] = gamma[:, 0, :]
            X[np.arange(splits[i], len(Y))[:, np.newaxis], I] = gamma
            
        elif i == 0:
            X[np.arange(0, splits[1])[:, np.newaxis], I] = gamma
        else:
            X[np.arange(splits[i], splits[i+1])[:, np.newaxis], I] = gamma

        if debug:
            print()
    return X

In [11]:
def OMP_verif(code):
    print(code)
    I = np.nonzero(code)
    print(I)
    print(code[I])

In [12]:
X, y = make_regression(n_samples = 50, n_features = 20, n_targets = 2, noise=4, random_state=0)

In [13]:
X.shape

(50, 20)

In [14]:
y.shape

(50, 2)

In [15]:
b_numpy = OMP(y.T, 3, X.T, debug=True)

(1,)
(50, 1)
(50,)
2449425.071243986
(2,)
(50, 2)
(50,)
1675390.9154018096
(3,)
(50, 3)
(50,)
1347560.0488303776
(1,)
(50, 1)
(50,)
1817892.1699581572
(2,)
(50, 2)
(50,)
1455231.954008666
(3,)
(50, 3)
(50,)
1157900.6749918351


In [16]:
OMP_verif(b_numpy)

[[  0.         132.14938371   0.           0.           0.
    0.           0.           0.          70.15323381   0.
    0.          98.72352887   0.           0.           0.
    0.           0.           0.           0.           0.        ]
 [  0.         136.86874232   0.           0.           0.
    0.           0.           0.          75.95811465   0.
    0.           0.           0.           0.           0.
    0.          67.35538119   0.           0.           0.        ]]
(array([0, 0, 0, 1, 1, 1]), array([ 1,  8, 11,  1,  8, 16]))
[132.14938371  70.15323381  98.72352887 136.86874232  75.95811465
  67.35538119]


In [17]:
b_scipy = OMP_2(y.T, 3, X.T, debug=True)

7.633996503399395
1.0000000000000002
[[ 7.6339965   3.7894417   1.49588522]
 [ 3.7894417  67.05616075 -2.05742687]
 [ 1.49588522 -2.05742687 65.66658662]]
(1,)
(50, 1)
(50,)
2449425.0712439856
(2, 50)
(50,)
(2,)
(50, 2)
(50,)
1675390.9154018096
(3, 50)
(50,)
(3,)
(50, 3)
(50,)
1347560.0488303776
6.3385975871569675
1.0000000000000004
[[ 6.33859759  3.7894417   1.49588522]
 [ 3.7894417  67.05616075  2.05742687]
 [ 1.49588522  2.05742687 65.66658662]]
(1,)
(50, 1)
(50,)
1817892.1699581577
(2, 50)
(50,)
(2,)
(50, 2)
(50,)
1455231.9540086659
(3, 50)
(50,)
(3,)
(50, 3)
(50,)
1157900.6749918351


In [18]:
OMP_verif(b_scipy)

[[  0.         132.14938371   0.           0.           0.
    0.           0.           0.          70.15323381   0.
    0.          98.72352887   0.           0.           0.
    0.           0.           0.           0.           0.        ]
 [  0.         136.86874232   0.           0.           0.
    0.           0.           0.          75.95811465   0.
    0.           0.           0.           0.           0.
    0.          67.35538119   0.           0.           0.        ]]
(array([0, 0, 0, 1, 1, 1]), array([ 1,  8, 11,  1,  8, 16]))
[132.14938371  70.15323381  98.72352887 136.86874232  75.95811465
  67.35538119]


In [19]:
b_scipy2 = OMP_3(y.T, 3, X.T, batch_size = 2, debug=False)

In [20]:
OMP_verif(b_scipy2)

[[  0.         132.14938371   0.           0.           0.
    0.           0.           0.          70.15323381   0.
    0.          98.72352887   0.           0.           0.
    0.           0.           0.           0.           0.        ]
 [  0.         136.86874232   0.           0.           0.
    0.           0.           0.          75.95811465   0.
    0.           0.           0.           0.           0.
    0.          67.35538119   0.           0.           0.        ]]
(array([0, 0, 0, 1, 1, 1]), array([ 1,  8, 11,  1,  8, 16]))
[132.14938371  70.15323381  98.72352887 136.86874232  75.95811465
  67.35538119]


In [21]:
# b_scipy3 = OMP_4(y.T, 3, X.T, debug=True)

In [22]:
# OMP_verif(b_scipy3)

In [23]:
# b_scipy4 = OMP_5(y.T, 3, X.T, debug=True)

In [24]:
# OMP_verif(b_scipy4)

In [25]:
np.allclose(b_scipy[np.nonzero(b_scipy)], b_numpy[np.nonzero(b_numpy)])

True

In [26]:
np.allclose(b_scipy2[np.nonzero(b_scipy2)], b_numpy[np.nonzero(b_numpy)])

True

In [27]:
# np.allclose(b_scipy3[np.nonzero(b_scipy3)], b_numpy[np.nonzero(b_numpy)])

In [28]:
# np.allclose(b_scipy4[np.nonzero(b_scipy4)], b_numpy[np.nonzero(b_numpy)])

## IT WORKS !!!

In [29]:
X, y = make_regression(n_samples = 50, n_features = 300, n_targets = 20_000, noise=4, random_state=0)

In [30]:
%timeit OMP(y.T, 1, X.T)

844 ms ± 14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
%timeit OMP_2(y.T, 1, X.T)

621 ms ± 65.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [32]:
%timeit OMP_3(y.T, 1, X.T, batch_size = 1)

1.03 s ± 87.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
%timeit OMP_3(y.T, 1, X.T, batch_size = 2)

519 ms ± 25 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [34]:
%timeit OMP_3(y.T, 1, X.T, batch_size = 4)

304 ms ± 6.19 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
%timeit OMP_3(y.T, 1, X.T, batch_size = 8)

180 ms ± 1.36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [36]:
%timeit OMP_3(y.T, 1, X.T, batch_size = 16)

111 ms ± 2.79 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [37]:
%timeit OMP_3(y.T, 1, X.T, batch_size = 32)

83.4 ms ± 1.21 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [38]:
%timeit OMP_3(y.T, 1, X.T, batch_size = 50)

67.9 ms ± 1.15 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [39]:
# %timeit OMP_4(y.T, 1, X.T)

In [40]:
# %timeit OMP_5(y.T, 1, X.T)

In [41]:
%load_ext line_profiler
from line_profiler import profile

In [42]:
np_OMP = profile(OMP)
%lprun -f np_OMP np_OMP(y.T, 1, X.T)

Timer unit: 1e-07 s

Total time: 1.52091 s
File: C:\Users\richa\AppData\Local\Temp\ipykernel_27324\3367727764.py
Function: OMP at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def OMP(Y, T_0, D, rng=42, debug=False):
     2                                               # loss = np.empty(num_iter)
     3                                               # rng = np.random.default_rng(rng)
     4                                           
     5         1        410.0    410.0      0.0      X = np.zeros((Y.shape[0], D.shape[0]))
     6                                           
     7                                               # gram = D @ D.T
     8                                               # cov = D @ Y.T
     9                                               # X = sparse_encode(Y, 
    10                                               #                   D, 
    11                                               # 

In [43]:
sci_OMP = profile(OMP_2)
%lprun -f sci_OMP sci_OMP(y.T, 1, X.T)

Timer unit: 1e-07 s

Total time: 0.815814 s
File: C:\Users\richa\AppData\Local\Temp\ipykernel_27324\3318475742.py
Function: OMP_2 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def OMP_2(Y, T_0, D, rng=42, debug=False):
     2                                               # loss = np.empty(num_iter)
     3                                               # rng = np.random.default_rng(rng)
     4                                           
     5         1        385.0    385.0      0.0      X = np.zeros((Y.shape[0], D.shape[0]))
     6         1       1417.0   1417.0      0.0      D_norm = np.linalg.norm(D, axis=1)
     7                                           
     8     20001     161813.0      8.1      2.0      for i, y in enumerate(Y):
     9     20000      50686.0      2.5      0.6          I = []
    10     20000     321249.0     16.1      3.9          D_I = np.zeros((T_0, D.shape[1]))
    11     20000     

In [44]:
sci_OMP2 = profile(OMP_3)
%lprun -f sci_OMP2 sci_OMP2(y.T, 1, X.T)

Timer unit: 1e-07 s

Total time: 0.742623 s
File: C:\Users\richa\AppData\Local\Temp\ipykernel_27324\3497261423.py
Function: OMP_3 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def OMP_3(Y, T_0, D, batch_size = 2, rng=42, debug=False):
     2                                               # loss = np.empty(num_iter)
     3                                               # rng = np.random.default_rng(rng)
     4                                           
     5         1        396.0    396.0      0.0      X = np.zeros((Y.shape[0], D.shape[0]))
     6         1       1000.0   1000.0      0.0      D_norm = np.linalg.norm(D, axis=1, keepdims=True)
     7                                           
     8         1        182.0    182.0      0.0      splits = np.arange(0, Y.shape[0], step=batch_size)
     9         1     407143.0 407143.0      5.5      Y_batches = np.split(Y, splits)[1:]
    10                         

In [45]:
%lprun -f sci_OMP2 sci_OMP2(y.T, 1, X.T, batch_size = 50)

Timer unit: 1e-07 s

Total time: 0.0747306 s
File: C:\Users\richa\AppData\Local\Temp\ipykernel_27324\3497261423.py
Function: OMP_3 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def OMP_3(Y, T_0, D, batch_size = 2, rng=42, debug=False):
     2                                               # loss = np.empty(num_iter)
     3                                               # rng = np.random.default_rng(rng)
     4                                           
     5         1        386.0    386.0      0.1      X = np.zeros((Y.shape[0], D.shape[0]))
     6         1        986.0    986.0      0.1      D_norm = np.linalg.norm(D, axis=1, keepdims=True)
     7                                           
     8         1        102.0    102.0      0.0      splits = np.arange(0, Y.shape[0], step=batch_size)
     9         1      18271.0  18271.0      2.4      Y_batches = np.split(Y, splits)[1:]
    10                        

In [46]:
# sci_OMP3 = profile(OMP_4)
# %lprun -f sci_OMP3 sci_OMP3(y.T, 1, X.T)

In [47]:
# sci_OMP4 = profile(OMP_5)
# %lprun -f sci_OMP4 sci_OMP4(y.T, 1, X.T)

In [48]:
X1, y1 = make_regression(n_samples = 50, n_features = 20, n_targets = 100, noise=4, random_state=0)

In [49]:
X2, y2 = make_regression(n_samples = 50, n_features = 20, n_targets = 100, noise=4, random_state=0)

In [50]:
X1.shape

(50, 20)

In [51]:
X2.shape

(50, 20)

In [52]:
X_12 = np.stack([X1, X2], axis=0)

In [53]:
X_12.shape

(2, 50, 20)

In [54]:
y_12 = np.stack([y1, y2], axis=0)

In [55]:
LA.lstsq(X_12, y_12)[0].shape

(2, 20, 100)

In [56]:
np.eye(3)[0][:, np.newaxis].shape

(3, 1)